In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import logging
os.environ["MALLOC_TRIM_THRESHOLD_"] = '0'

import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import xrft
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("figure", figsize=(12,10))
plt.rc("font", size=14)

from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, silence_logs='error')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36543 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:36543/status,
Dashboard: http://137.129.155.67:36543/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: inproc://137.129.155.67/51440/1,Workers: 1
Dashboard: http://137.129.155.67:36543/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: inproc://137.129.155.67/51440/4,Total threads: 4
Dashboard: http://137.129.155.67:41787/status,Memory: 78.61 GiB
Nanny: None,


In [16]:
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/OLR/'
year = 2009

data_OLR = xr.open_mfdataset(indir_data_FILTERED +  '*' + str(year) + '.nc')
data_OLR = data_OLR.sel(lat = slice(-15,15))

indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/TCWV/'
data_TCWV = xr.open_mfdataset(indir_data_FILTERED +  '*' + str(year) + '.nc')
data_TCWV = data_TCWV.reindex(lat=list(reversed(data_TCWV.lat)))
data_TCWV = data_TCWV.sel(lat = slice(-15,15))

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key

In [17]:
data_OLR

<xarray.Dataset>
Dimensions:     (time: 2920, lat: 30, lon: 360)
Coordinates:
  * time        (time) datetime64[ns] 2009-01-01 ... 2009-12-31T21:00:00
  * lat         (lat) float32 -14.5 -13.5 -12.5 -11.5 ... 11.5 12.5 13.5 14.5
  * lon         (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Data variables:
    OLR_Kelvin  (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_Rossby  (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_MJO     (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_MRG     (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_EIG     (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_WIG     (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_TD      (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>
    OLR_LF      (time, lat, lon) float64 dask.array<chunksize=(2920, 30, 360), meta=np.ndarray>

In [18]:
data_TCWV = data_TCWV.isel(lat = slice(2,None,4), lon = slice(2,None,4))

In [21]:
whatbhat = xrft.cross_spectrum(data_OLR['OLR_Kelvin'].chunk({'time':1}), data_TCWV['TCWV_Kelvin'].chunk({'time':1}),
                                   dim=['lat','lon'], detrend='linear', window=True, density=False).compute()
whatbhat

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/xrft/xrft.py:49: FutureWarning: Please provide the name of window adhering to scipy.signal.windows. The boolean option will be deprecated in future releases.
  warnings.warn(
/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/xrft/xrft.py:49: FutureWarning: Please provide the name of window adhering to scipy.signal.windows. The boolean option will be deprecated in future releases.
  warnings.warn(


<xarray.DataArray (time: 2920, freq_lat: 30, freq_lon: 360)>
array([[[-4.07190545e-29-5.28235470e-28j,
          1.64947310e-27-3.39227821e-28j,
          1.18542573e-27-7.38089947e-28j, ...,
         -5.57913086e-29-6.22214074e-28j,
          9.54078241e-29+1.30433500e-27j,
          1.18542665e-27-1.17842198e-28j],
        [ 1.22215170e-27+1.52467570e-27j,
         -1.48931888e-27+8.00354973e-28j,
         -8.30760822e-28+1.50358087e-28j, ...,
          9.53150596e-29+3.89497388e-28j,
         -9.43505169e-28+1.96002895e-27j,
          1.02857390e-27+2.60098744e-27j],
        [ 9.56849206e-28+2.31169928e-27j,
         -2.17993257e-27-1.42417062e-27j,
         -8.02413628e-28-1.54479157e-27j, ...,
         -1.34553422e-27+4.68600643e-27j,
         -1.14527899e-28+6.37438969e-27j,
          1.64472315e-27+5.01397094e-27j],
        ...,
        [-4.95967386e-28-7.86619477e-30j,
...
          3.01336022e-27-7.49383351e-28j],
        ...,
        [-1.43284859e-27-6.68656900e-27j,
         -1.22209484e-27-6.07705743e-28j,
          1.47554666e-28-1.15908341e-27j, ...,
         -2.10111940e-27+1.12992739e-26j,
          1.82108982e-28+3.90641794e-27j,
         -5.14946614e-27-5.50213472e-27j],
        [ 7.24627961e-28+3.78821249e-29j,
          1.14161245e-27+7.84891116e-28j,
          1.82159474e-27-1.38750338e-27j, ...,
          5.24871020e-27+4.67585893e-27j,
          3.14603539e-27+1.33247627e-26j,
         -1.89361600e-27+5.56386759e-27j],
        [-9.45639744e-28-3.96956245e-28j,
          1.07738731e-27+9.58146799e-27j,
          3.34625153e-27+3.16396839e-28j, ...,
         -2.93288419e-27+1.06179990e-27j,
         -7.71432134e-27+3.58215136e-27j,
         -3.60879435e-27-8.43954678e-29j]]])
Coordinates:
  * time      (time) datetime64[ns] 2009-01-01 ... 2009-12-31T21:00:00
  * freq_lat  (freq_lat) float64 -0.5 -0.4667 -0.4333 -0.4 ... 0.4 0.4333 0.4667
  * freq_lon  (freq_lon) float64 -0.5 -0.4972 -0.4944 ... 0.4917 0.4944 0.4972